In [18]:
import time
import numpy as np

In [ ]:
x1 = [9, 2, 5, 0, 0, 7, 5, 0, 0, 0, 9, 2, 5, 0, 0]
x2 = [9, 2, 2, 9, 0, 9, 2, 5, 0, 0, 9, 2, 5, 0, 0]
W =  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

### VECTORIZED DOT PRODUCT OF VECTORS ###
tic = time.process_time()
dot = np.dot(x1,x2)
toc = time.process_time()
print ("dot = " + str(dot) + "\n ----- Computation time = " + str(1000 * (toc - tic)) + "ms")

### VECTORIZED OUTER PRODUCT ###
tic = time.process_time()
outer = np.outer(x1,x2)
toc = time.process_time()
print ("outer = " + str(outer) + "\n ----- Computation time = " + str(1000 * (toc - tic)) + "ms")

### VECTORIZED ELEMENTWISE MULTIPLICATION ###
tic = time.process_time()
mul = np.multiply(x1,x2)
toc = time.process_time()
print ("elementwise multiplication = " + str(mul) + "\n ----- Computation time = " + str(1000*(toc - tic)) + "ms")

### VECTORIZED GENERAL DOT PRODUCT ###
tic = time.process_time()
dot = np.dot(x1,x2)
toc = time.process_time()
print ("gdot = " + str(dot) + "\n ----- Computation time = " + str(1000 * (toc - tic)) + "ms")

## **Logistic Regression**

In [ ]:
import numpy as np

In [2]:
def sigmoid(x):
  s = 1 / ( 1 + np.exp(-x))
  return s

In [3]:
def initialize_with_zeros(dim):
  w = np.zeros((dim,1))
  b = 0
  return w,b

In [4]:
def propagate(w, b, X, Y):
  m = X.shape[1]
  A = sigmoid(np.dot(w.T,X) + b)
  J = -1/m * (np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A)))

  dw = 1/m * (np.dot(X , (A - Y).T))
  db = 1/m * (np.sum(A - Y))

  assert(dw.shape == w.shape)
  assert(db.dtype == float)
  J = np.squeeze(J)
  assert(J.shape == ())

  grads = {"dw" : dw, "db" : db}

  return grads,J

In [5]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
  costs = []

  for i in range(num_iterations):

    grads, J = propagate(w, b, X, Y)
    dw = grads["dw"]
    db = grads["db"]

    w = w - learning_rate * dw
    b = b - learning_rate * db

    if i % 100 == 0:
      costs.append(J)

    #Print the cost every 100 training iterations
    if print_cost and i % 100 == 0:
      print ("Cost after iteration %i: %f" %(i, J))


  params = {"w": w, "b": b}

  grads = {"dw": dw, "db": db}

  return params, grads, costs

In [6]:
def predict(w, b, X):
  m = X.shape[1]
  Y_prediction = np.zeros((1,m))
  w = w.reshape(X.shape[0], 1)
  A = sigmoid(np.dot(w.T,X) + b)


  for i in range(A.shape[1]):
    if A[0][i] <= 0.5:
      Y_prediction[0][i] = 0
    else:
      Y_prediction[0][i] = 1


  assert(Y_prediction.shape == (1, m))

  return Y_prediction

In [7]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):

  w, b = initialize_with_zeros(X_train.shape[0])

  parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)

  w = parameters["w"]
  b = parameters["b"]

  Y_prediction_test = predict(w, b, X_test)
  Y_prediction_train = predict(w, b, X_train)

  print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
  print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))


  d = {"costs": costs,
      "Y_prediction_test": Y_prediction_test,
      "Y_prediction_train" : Y_prediction_train,
      "w" : w,
      "b" : b,
      "learning_rate" : learning_rate,
      "num_iterations": num_iterations}

  return d

## **One Hidden Layer NN**

In [8]:
def layer_sizes(X, Y):
  """
  Arguments:
  X -- input dataset of shape (input size, number of examples)
  Y -- labels of shape (output size, number of examples)

  Returns:
  n_x -- the size of the input layer
  n_h -- the size of the hidden layer
  n_y -- the size of the output layer
  """

  n_x = X.shape[0]
  n_h = 4
  n_y = Y.shape[0]

  return (n_x, n_h, n_y)

In [9]:
def initialize_parameters(n_x, n_h, n_y):
  np.random.seed(2)

  W1 = np.random.randn(n_h, n_x)
  b1 = np.zeros((n_h,1))
  W2 = np.random.randn(n_y,n_h)
  b2 = np.zeros((n_y,1))

  parameters = {"W1": W1,"b1": b1,"W2": W2,"b2": b2}

  return parameters

In [10]:
def forward_propagation(X, parameters):

  W1 = parameters["W1"]
  b1 = parameters["b1"]
  W2 = parameters["W2"]
  b2 = parameters["b2"]

  Z1 = np.dot(W1,X) + b1
  A1 = np.tanh(Z1)
  Z2 = np.dot(W2,A1) + b2
  A2 = sigmoid(Z2)

  assert(A2.shape == (1, X.shape[1]))

  cache = {"Z1": Z1,"A1": A1,"Z2": Z2,"A2": A2}

  return A2, cache

In [11]:
def compute_cost(A2, Y, parameters):

  m = Y.shape[1]

  logprobs = np.multiply(np.log(A2),Y) + np.multiply(np.log(1 - A2),1 - Y)
  cost = -1/m * np.sum(logprobs)

  cost = float(np.squeeze(cost))

  assert(isinstance(cost, float))

  return cost

In [12]:
def backward_propagation(parameters, cache, X, Y):

  m = X.shape[1]

  W1 = parameters["W1"]
  W2 = parameters["W2"]

  A1 = cache["A1"]
  A2 = cache["A2"]

  dZ2 = A2 - Y
  dW2 = 1/m * (np.dot(dZ2, A1.T))
  db2 = 1/m * (np.sum(dZ2, axis = 1, keepdims =  True))
  dZ1 = (np.dot(W2.T, dZ2)) * (1 - np.power(A1, 2))
  dW1 = 1/m * (np.dot(dZ1,X.T))
  db1 = 1/m * (np.sum(dZ1, axis = 1, keepdims =  True))

  grads = {"dW1": dW1,"db1": db1,"dW2": dW2,"db2": db2}

  return grads

In [13]:
def update_parameters(parameters, grads, learning_rate = 1.2):

  W1 = parameters["W1"]
  b1 = parameters["b1"]
  W2 = parameters["W2"]
  b2 = parameters["b2"]

  dW1 = grads["dW1"]
  db1 = grads["db1"]
  dW2 = grads["dW2"]
  db2 = grads["db2"]

  W1 = W1 - learning_rate * dW1
  b1 = b1 - learning_rate * db1
  W2 = W2 - learning_rate * dW2
  b2 = b2 - learning_rate * db2

  parameters = {"W1": W1,"b1": b1,"W2": W2,"b2": b2}

  return parameters

In [14]:
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):

  np.random.seed(3)
  n_x = layer_sizes(X, Y)[0]
  n_y = layer_sizes(X, Y)[2]

  parameters = initialize_parameters(n_x, n_h, n_y)

  for i in range(0, num_iterations):

    A2, cache =  forward_propagation(X, parameters)

    cost = compute_cost(A2, Y, parameters)

    grads =  backward_propagation(parameters, cache, X, Y)

    parameters = update_parameters(parameters, grads, learning_rate = 1.2)

    if print_cost and i % 1000 == 0:
      print ("Cost after iteration %i: %f" %(i, cost))

  return parameters

In [15]:
def predict(parameters, X):

  A2, cache = forward_propagation(X, parameters)
  predictions = (A2 > 0.5)

  return predictions